# Multi-label tabular with AutoGluon (binary one-vs-rest)

Using two binary TabularPredictor models as a workaround because MultiLabelPredictor is unavailable on this Python version.

In [1]:
from autogluon.tabular import TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

DATA_PATH = Path("Assignment 6/AutoGluon/extra-credit/multilabel/data/train_multi.csv")
LABEL_COLS = ["class_>50K", "class_<=50K"]
TIME_LIMIT = 30

In [2]:
df = pd.read_csv(DATA_PATH)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["class"])
print("Train", train_df.shape, "Val", val_df.shape)
train_df.head()

Train (31258, 17) Val (7815, 17)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,class_>50K,class_<=50K
30271,21,Private,138768,Some-college,10,Never-married,Sales,Other-relative,White,Male,0,0,40,United-States,<=50K,0,1
36316,59,Private,296253,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Female,0,0,40,United-States,<=50K,0,1
2694,50,Private,158948,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,3411,0,40,United-States,<=50K,0,1
39056,39,Private,185084,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,55,United-States,<=50K,0,1
30189,46,Private,295334,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,60,United-States,>50K,1,0


In [3]:
results = {}
pred_frames = []
for label in LABEL_COLS:
    path = Path("Assignment 6/AutoGluon/extra-credit/multilabel/ag_multilabel_models") / label
    predictor = TabularPredictor(label=label, path=path, problem_type="binary", eval_metric="f1")
    predictor.fit(train_df, tuning_data=val_df, time_limit=TIME_LIMIT, presets="medium_quality_faster_train")
    metrics = predictor.evaluate(val_df)
    results[label] = metrics
    preds = predictor.predict(val_df)
    prob = predictor.predict_proba(val_df)[1]
    pred_frames.append(pd.DataFrame({f"pred_{label}": preds, f"prob_{label}": prob}))
combined = pd.concat(pred_frames, axis=1)
combined.head()

Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.


Verbosity: 2 (Standard Logging)


=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.1.0: Mon Oct 20 19:32:41 PDT 2025; root:xnu-12377.41.6~2/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       3.75 GB / 16.00 GB (23.4%)
Disk Space Avail:   268.95 GB / 460.43 GB (58.4%)


Presets specified: ['medium_quality_faster_train']


Beginning AutoGluon training ... Time limit = 30s


...

	0.48s	 = Training   runtime


	0.0s	 = Validation runtime


AutoGluon training complete, total runtime = 18.33s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 192862.3 rows/s (7815 batch size)


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("Assignment 6/AutoGluon/extra-credit/multilabel/ag_multilabel_models/class_<=50K")


,pred_class_>50K,prob_class_>50K,pred_class_<=50K,prob_class_<=50K
29579,0,0.0,1,1.0
26121,0,0.0,1,1.0
25570,0,0.0,1,1.0
16496,0,0.0,1,1.0
33535,0,0.0,1,1.0


In [4]:
results

{'class_>50K': {'f1': 1.0,
  'accuracy': 1.0,
  'balanced_accuracy': 1.0,
  'mcc': 1.0,
  'roc_auc': 1.0,
  'precision': 1.0,
  'recall': 1.0},
 'class_<=50K': {'f1': 1.0,
  'accuracy': 1.0,
  'balanced_accuracy': 1.0,
  'mcc': 1.0,
  'roc_auc': 1.0,
  'precision': 1.0,
  'recall': 1.0}}